#### -----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.
##### THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN  THE  SOFTWARE.<br>-----------------------------------------------------------------------------

In [ ]:
import time
from arena_api.system import system

##### Chunk Data: CRC Validation
>    This example demonstrates the use of chunk data to verify data
    through a Cyclical Redundancy Check (or CRC for short). CRCs are
    meant to check the validity of sent data. It is performed by doing
    a series of calculations on the raw data before and after it is
    sent. If the resultant integer values match, then it is safe to
    assume the integrity of the data.

In [ ]:
"""
This code block waits for the user to connect a device before raising
an exception
"""

tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)\n')
        device = devices[0]
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

##### Configure devices and store initial values
> These initial values are restored to the device after the example is completed

In [ ]:
device = devices[0]
print(f'Device used in the example:\n\t{device}')

nodemap = device.nodemap
tl_stream_nodemap = device.tl_stream_nodemap

# Store initial acquisition mode and set it to Continuous
initial_acquisition_mode = nodemap.get_node("AcquisitionMode").value
nodemap.get_node("AcquisitionMode").value = "Continuous"

# Enable stream auto negotiate packet size
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True

# Enable stream packet resend
tl_stream_nodemap['StreamPacketResendEnable'].value = True

##### Prep device to send chunk data with the buffer

In [ ]:
initial_chunk_mode_active = nodemap.get_node("ChunkModeActive").value
print('Activating chunk data on device')
nodemap.get_node('ChunkModeActive').value = True

# Use CRC chunk: unnecessary, any chunk works
initial_chunk_selector = nodemap.get_node('ChunkSelector').value
print(f"\tSetting 'ChunkSelector' node value to 'CRC'")
nodemap.get_node("ChunkSelector").value = "CRC"

initial_chunk_enable = nodemap.get_node('ChunkEnable').value
print(f"\tEnabling CRC by setting 'ChunkEnable' node value to 'True'")
nodemap.get_node("ChunkEnable").value = True

##### Retrieve and validate chunk data using CRC

##### Grab images -------------------------------------------------------------
> - Starting the stream allocates buffers, which can be passed in as
 an argument (default: 10), and begins filling them with data.
 Buffers must later be requeued to avoid memory leaks.<br>
> - 'Device.get_buffer()' with no arguments returns one buffer(NOT IN A LIST)<br>
> - 'Device.get_buffer(30)' returns 30 buffers(IN A LIST)<br>
> - 'Device.requeue_buffer()' takes a buffer or many buffers in a list or tuple

In [ ]:
with device.start_stream():
    print(f'Stream started with 10 buffers')
    
    print('\tGet chunk data buffer(s)')

    # This would timeout or returns all of the 10 buffers
    chunk_data_buffers = device.get_buffer(number_of_buffers=10)
    print(f'\t{len(chunk_data_buffers)} chunk data buffers received')

    '''
    Confirm that each chunk is complete, then validate the buffer's CRC
    '''
    for buffer_index, chunk_data_buffer in enumerate(chunk_data_buffers):
        if chunk_data_buffer.is_incomplete:
            print(f'\t\t---------------------------------------------')
            print(f'\t\tChunk data buffer{buffer_index} is incomplete')
            print(f'\t\t---------------------------------------------')
            # Continue
        try:
            '''
            Buffer.is_valid_crc: compares calculated CRC value to the chunk CRC value.
                This only works with buffers retrieved in chunk mode.
            '''
            print(f"\t\tChunk data CRC is valid: {chunk_data_buffer.is_valid_crc}")
        except ValueError:
            print(f'\t\t\tFailed to get chunks')
            print(f'\t\t---------------------------------------------')
            continue

    device.requeue_buffer(chunk_data_buffers)

##### Clean up ----------------------------------------------------------------
> - Restore initial values to the device.
> - Destroy device. This call is optional and will automatically be
  called for any remaining devices when the system module is unloading.

In [ ]:
nodemap.get_node("ChunkEnable").value = initial_chunk_enable
nodemap.get_node("ChunkSelector").value = initial_chunk_selector
nodemap.get_node("ChunkModeActive").value = initial_chunk_mode_active

nodemap.get_node("AcquisitionMode").value = initial_acquisition_mode

system.destroy_device()
print('Destroyed all created devices')